In [ ]:
import random
import re
import numpy as np

# Parameters from the research paper
alpha = 1.0  # Influence of pheromone
rho = 0.99   # Pheromone evaporation rate
tau_min = 0.01  # Minimum pheromone trail
tau_max = 6.0   # Maximum pheromone trail
num_ants = 30
num_iterations = 300

# Read adjacency matrix from file
my_file = open('/content/sample_data/5.clq', 'r')  # Load file
clique_line = my_file.readline()      # read first line
node_edge_line = my_file.readline()   # read second line

clique_size = int(re.findall(r'\b\d+\b', clique_line)[0])  # extract first number as max clique size
node = int(re.findall(r'\b\d+\b', node_edge_line)[0])      # extract first number as number of nodes

adj_matrix = [[0] * node for _ in range(node)]  # Create 2D matrix initialized with zeros

while True:  # Initialize adjacency matrix
    next_line = my_file.readline().strip()
    if not next_line:
        break  # Exit loop if no more lines are left
    u = int(re.findall(r'\b\d+\b', next_line)[0]) - 1  # 1 less numbered
    v = int(re.findall(r'\b\d+\b', next_line)[1]) - 1  # 1 less numbered
    adj_matrix[u][v] = 1  # 1 indicates there is an edge
    adj_matrix[v][u] = 1  # 1 indicates there is an edge

# Pheromone matrix (initialize with tau_max)
pheromone = [[tau_max for _ in range(node)] for _ in range(node)]

# Function to check if a clique is valid
def is_valid_clique(clique):
    for i in range(len(clique)):
        for j in range(i + 1, len(clique)):
            if adj_matrix[clique[i]][clique[j]] == 0:
                return False
    return True

# Local Search: Improve clique by exploring neighbors
def local_search(clique):
    improved = True
    while improved:
        improved = False
        for v in range(node):
            if v not in clique and all(adj_matrix[v][u] == 1 for u in clique):
                clique.append(v)
                improved = True
    return clique

# Ant class
class Ant:
    def __init__(self):
        self.clique = []

    def construct_solution(self):
        # Start from a random node
        current_node = random.randint(0, node - 1)
        self.clique = [current_node]

        # Expand the clique
        while True:
            candidates = []
            for next_node in range(node):
                if next_node not in self.clique and all(adj_matrix[next_node][n] == 1 for n in self.clique):
                    candidates.append(next_node)

            if not candidates:
                break

            # Calculate probabilities based on pheromone and heuristic
            probabilities = []
            for candidate in candidates:
                pheromone_value = sum(pheromone[n][candidate] for n in self.clique)  # Pheromone contribution
                probabilities.append(pheromone_value ** alpha)

            total_probability = sum(probabilities)
            probabilities = [p / total_probability for p in probabilities]

            # Select next node probabilistically
            next_node = random.choices(candidates, probabilities)[0]
            self.clique.append(next_node)

        # Apply local search to improve the solution
        self.clique = local_search(self.clique)

# ACO algorithm
def aco_max_clique():
    best_clique = []

    for iteration in range(num_iterations):
        ants = [Ant() for _ in range(num_ants)]
        iteration_best_clique = []

        # Each ant constructs a solution
        for ant in ants:
            ant.construct_solution()
            if is_valid_clique(ant.clique) and len(ant.clique) > len(iteration_best_clique):
                iteration_best_clique = ant.clique

        # Update the global best solution
        if len(iteration_best_clique) > len(best_clique):
            best_clique = iteration_best_clique

        # Pheromone evaporation
        for i in range(node):
            for j in range(node):
                pheromone[i][j] = max(tau_min, pheromone[i][j] * rho)

        # Update pheromones based on the best clique found in this iteration
        if is_valid_clique(iteration_best_clique):
            clique_size = len(iteration_best_clique)
            for i in range(clique_size):
                for j in range(i + 1, clique_size):
                    pheromone[iteration_best_clique[i]][iteration_best_clique[j]] += 1.0 / clique_size
                    pheromone[iteration_best_clique[j]][iteration_best_clique[i]] += 1.0 / clique_size
                    pheromone[iteration_best_clique[i]][iteration_best_clique[j]] = min(
                        pheromone[iteration_best_clique[i]][iteration_best_clique[j]], tau_max)

    return best_clique

# Run ACO
best_clique = aco_max_clique()
print(f"Max Clique Size (Benchmark): {clique_size}")
print(f"Max Clique Size Found: {len(best_clique)}")
print(f"Max Clique Nodes: {best_clique}")